In [1]:
#####Latent Dirichlet Allocation Model#####
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt
import numpy.matlib
import gensim
import itertools
from numpy.random import *
from scipy import optimize
from scipy import sparse
import seaborn as sns

C:\Users\sana\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
##多項分布の乱数を生成する関数
def rmnom(pr, n, k, no):
    z_id = np.argmax((np.cumsum(pr, axis=1) > np.random.rand(n).reshape(n, 1)), axis=1)
    Z = sparse.coo_matrix((np.repeat(1, n), (no, np.array(z_id))), shape=(n, k))   #スパース行列の設定
    return Z

In [3]:
####データの発生####
##データの設定
k = 15   #トピック数
d = 3000   #文書数
v  = 1000   #語彙数
w = np.random.poisson(np.random.gamma(27.5, 1/0.2, d))   #1文書あたりの単語数
f = sum(w)   #総単語数
vec_f = np.array(range(f))
vec_k = np.array(range(k))
vec_v = np.array(range(v))

In [4]:
##IDとインデックスを設定
#IDの設定
d_id = np.repeat(range(d), w)
pt_id = np.array(list(itertools.chain(*[np.array(range(w[i]), dtype="int") for i in range(d)])))

#インデックスの設定
index = np.array(range(f))
d_list = [i for i in range(d)]
for i in range(d):
    d_list[i] = index[d_id==i]
d_dt = sparse.coo_matrix((np.repeat(1, f), (range(f), d_id)), shape=(f, d)).tocsr()   #スパース行列の設定
d_dt_T = d_dt.T

In [5]:
##すべての単語が生成されればbreak
rp = 0
while True:
    rp = rp + 1
    print(rp)
    
    ##パラメータの設定
    #ディレクリ分布のパラメータの設定
    alpha0 = np.repeat(0.15, k)   #文書のディレリ事前分布のパラメータ
    alpha1 = np.repeat(0.05, v)   #単語のディレクリ事前分布のパラメータ

    #ディレクリ分布からパラメータを生成
    theta = np.random.dirichlet(alpha0, d)   #トピック分布を生成
    phi = np.random.dirichlet(alpha1, k)    #単語分布を生成

    #単語出現確率が低いトピックを入れ替える
    index = np.array(range(v))[np.max(phi, axis=0) < (k*5)/f]
    for j in range(index.shape[0]):
        phi[np.dot(np.random.multinomial(1, np.random.dirichlet(alpha0, 1).reshape(-1), 1), range(k)), index[j]] = (k*5)/f
    phit = phi; thetat = theta
    
    ##トピックと単語を生成
    #生成したデータの格納用配列
    WX = np.zeros((d, v), dtype="int")
    Z = np.zeros((f, k), dtype="int")
    wd = np.repeat(0, f)

    for i in range(d):
        #多項分布からトピックを生成
        z = np.random.multinomial(1, theta[i, ], w[i])
        z_vec = np.dot(z, vec_k)

        #トピックから単語を生成
        word = np.array(rmnom(phi[z_vec, ], w[i], v, np.array(range(w[i]))).todense())
        word_vec = np.dot(word, vec_v)

        #データを格納
        Z[d_list[i], ] = z
        wd[d_list[i]] = word_vec
        WX[i, ] = np.sum(word, axis=0)

    #すべての単語の出現があればbreak
    if(np.min(np.sum(WX, axis=0)) > 0):
        break

1
2


In [6]:
#単語のインデックスとスパース行列を作成
index = np.array(range(f))
w_list = [j for j in range(v)]
for i in range(v):
    w_list[i] = index[wd==i]
sparse_data = sparse.coo_matrix((np.repeat(1, f), (range(f), wd)), shape=(f, v)).tocsr()   #スパース行列の設定
sparse_data_T = sparse_data.T

In [7]:
#トピック尤度と負担率を計算する関数
def LLho(theta, phi, d_id, wd, f, k):
    Lho = theta[d_id, ] * (phi.T)[wd, ]
    topic_rate = Lho / np.sum(Lho, axis=1).reshape(f, 1)
    return Lho, topic_rate

In [8]:
##アルゴリズムの設定
R = 2000
keep = 2
burnin = int(500/keep)
iter = 0 
disp =10

In [9]:
##事前分布の設定
alpha1 = 0.1 
beta1 = 0.1

In [10]:
##パラメータの設定
#パラメータの真値
theta = thetat
phi = phit

In [11]:
#パラメータの初期値
phi = np.random.dirichlet(np.repeat(1, v), k)
theta = np.random.dirichlet(np.repeat(1, k), d)

In [12]:
##パラメータの保存用配列
THETA = np.zeros((d, k, R//keep), dtype='int')
PHI = np.zeros((k, v, R//keep), dtype='int')
SEG = np.zeros((f, k), dtype='int')

In [13]:
##対数尤度の基準値
#ユニグラムモデルの対数尤度
LLst = np.sum(np.dot(WX, np.log(np.sum(WX, axis=0) / f)))

#真値での対数尤度
Lho = thetat[d_id, ] * (phit.T)[wd, ]
LLbest = np.sum(np.log(np.sum(Lho, axis=1)))

In [14]:
####ギブスサンプリングでパラメータをサンプリング####
for rp in range(R):
    
    ##単語トピックを生成
    #トピック選択確率を設定
    out = LLho(theta, phi, d_id, wd, f, k)    
    topic_rate = out[1]

    #多項分布からトピックを生成
    Z = rmnom(topic_rate, f, k, vec_f).tocsr()


    ##パラメータをサンプリング
    #トピック分布をサンプリング
    wsum = np.array(np.dot(d_dt_T, Z).todense()) + alpha1
    for i in range(d):
        theta[i, :] = np.random.dirichlet(wsum[i, ], 1)

    #単語分布をサンプリング
    vsum = np.array(np.dot(sparse_data_T, Z).T.todense()) + beta1
    for j in range(k):
        phi[j, :] = np.random.dirichlet(vsum[j, ], 1)


    ##サンプリング結果の格納とパラメータの表示
    #サンプリング結果の保存
    if rp%keep==0:
        mkeep = rp//keep
        THETA[:, :, mkeep] = theta
        PHI[:, :, mkeep] = phi

    #バーンインを超えたらトピックを格納
    if (rp >= burnin) & (rp%keep==0):
        SEG += Z.todense() 

    #パラメータの表示
    if rp%disp==0:
        LL = np.sum(np.log(np.sum(theta[d_id, ] * (phi.T)[wd, ], axis=1)))   #対数尤度を計算
        print(rp)
        print(np.round([LL, LLst, LLbest], 1))

0
[-2634545.9 -2628157.4 -2275809.8]
10
[-2522011.2 -2628157.4 -2275809.8]
20
[-2407880.7 -2628157.4 -2275809.8]
30
[-2339933.5 -2628157.4 -2275809.8]
40
[-2310274.8 -2628157.4 -2275809.8]
50
[-2300938.1 -2628157.4 -2275809.8]
60
[-2297241.  -2628157.4 -2275809.8]
70
[-2295179.1 -2628157.4 -2275809.8]
80
[-2291341.2 -2628157.4 -2275809.8]
90
[-2284525.5 -2628157.4 -2275809.8]
100
[-2279812.5 -2628157.4 -2275809.8]
110
[-2277437.7 -2628157.4 -2275809.8]
120
[-2276958.7 -2628157.4 -2275809.8]
130
[-2276784.9 -2628157.4 -2275809.8]
140
[-2276709.9 -2628157.4 -2275809.8]
150
[-2276501.4 -2628157.4 -2275809.8]
160
[-2276670.9 -2628157.4 -2275809.8]
170
[-2276800.1 -2628157.4 -2275809.8]
180
[-2276603.8 -2628157.4 -2275809.8]
190
[-2276855.7 -2628157.4 -2275809.8]
200
[-2276865.5 -2628157.4 -2275809.8]
210
[-2276719.4 -2628157.4 -2275809.8]
220
[-2276379.2 -2628157.4 -2275809.8]
230
[-2276544.9 -2628157.4 -2275809.8]
240
[-2276788.7 -2628157.4 -2275809.8]
250
[-2276686.2 -2628157.4 -2275809.

[-2629969.8 -2623927.1 -2277977.6]
